In [2]:
import pandas as pd    
import gzip
import os

def getListOfFiles(dirName):
    """Walks a directory to return a list of paths to all the files in it and it's
    sub-directories.
    Arguments:
    dirName (string): path to directory to walk
    """
    list_of_files = os.listdir(dirName)
    all_files = list()
    for entry in list_of_files:
        full_path = os.path.join(dirName, entry)
        if os.path.isdir(full_path):
            all_files = all_files + getListOfFiles(full_path)
        else:
            all_files.append(full_path)
    return all_files

In [4]:
# start by getting a file path to each language file
fp_list = getListOfFiles('final/jsonl/valid')
fp_list
del fp_list[0]

In [5]:
# then we read the data into Dataframes for analysis and preparation
df_list = []
for fp in fp_list:
    df_list.append(pd.read_json(path_or_buf=fp, lines=True))


In [6]:
df_list[0].head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition
0,openai/baselines,baselines/deepq/deepq.py,learn,"def learn(env,\n network,\n ...",python,"def learn(env,\n network,\n ...","[def, learn, (, env, ,, network, ,, seed, =, N...",Train a deepq model.\n\n Parameters\n --...,"[Train, a, deepq, model, .]",3301089b48c42b87b396e246ea3f56fa4bfc9678,https://github.com/openai/baselines/blob/33010...,valid
1,openai/baselines,baselines/deepq/deepq.py,ActWrapper.save_act,"def save_act(self, path=None):\n """"""Sav...",python,"def save_act(self, path=None):\n """"""Sav...","[def, save_act, (, self, ,, path, =, None, ), ...",Save model to a pickle located at `path`,"[Save, model, to, a, pickle, located, at, path]",3301089b48c42b87b396e246ea3f56fa4bfc9678,https://github.com/openai/baselines/blob/33010...,valid
2,openai/baselines,baselines/common/models.py,nature_cnn,"def nature_cnn(unscaled_images, **conv_kwargs)...",python,"def nature_cnn(unscaled_images, **conv_kwargs)...","[def, nature_cnn, (, unscaled_images, ,, *, *,...",CNN from Nature paper.,"[CNN, from, Nature, paper, .]",3301089b48c42b87b396e246ea3f56fa4bfc9678,https://github.com/openai/baselines/blob/33010...,valid
3,openai/baselines,baselines/common/models.py,mlp,"def mlp(num_layers=2, num_hidden=64, activatio...",python,"def mlp(num_layers=2, num_hidden=64, activatio...","[def, mlp, (, num_layers, =, 2, ,, num_hidden,...",Stack of fully-connected layers to be used in ...,"[Stack, of, fully, -, connected, layers, to, b...",3301089b48c42b87b396e246ea3f56fa4bfc9678,https://github.com/openai/baselines/blob/33010...,valid
4,openai/baselines,baselines/common/models.py,lstm,"def lstm(nlstm=128, layer_norm=False):\n """"...",python,"def lstm(nlstm=128, layer_norm=False):\n """"...","[def, lstm, (, nlstm, =, 128, ,, layer_norm, =...",Builds LSTM (Long-Short Term Memory) network t...,"[Builds, LSTM, (, Long, -, Short, Term, Memory...",3301089b48c42b87b396e246ea3f56fa4bfc9678,https://github.com/openai/baselines/blob/33010...,valid


In [7]:
# check for duplicates and remove them if necessary

number_of_dps = 0

for df in df_list:
    number_of_dps += len(df)
print("number of data points ====> " + str(number_of_dps))
after = 0

for df in df_list:
    df.drop_duplicates(["docstring", "original_string"])
    after += len(df)
print("number of data points after processing ====> " + str(after))

number of data points ====> 23107
number of data points after processing ====> 23107


In [8]:
# remove all docstrings that are less than five words

after = 0
dfs = []
for df in df_list:
    df = df[df["docstring_tokens"].map(lambda x: len(x) > 4)]
    dfs.append(df)
    after += len(df)
print("number of data points after processing ====> " + str(after))

number of data points after processing ====> 20854


In [9]:
# remove all the __init__() function calls

after = 0
df_list = []
for df in dfs:
    df = df[df["docstring_tokens"].map(lambda x: "__init__" not in x)]
    df_list.append(df)
    after += len(df)
print("number of data points after processing ====> " + str(after))


number of data points after processing ====> 20835


In [10]:
# sort the data in ascending order of function tokens

python_df = pd.concat(df_list)
python_df["len_tok"] = python_df["code_tokens"].map(lambda x: len(x))
python_df.sort_values(by="len_tok", inplace=True)

del python_df["partition"]
del python_df["url"]
del python_df["language"]
del python_df["func_name"]
del python_df["repo"]
del python_df["path"]
del python_df["original_string"]
del python_df["code_tokens"]
del python_df["docstring_tokens"]
del python_df["sha"]
del python_df["len_tok"]

In [11]:
python_df.head()

python_df.to_pickle("python_valid.pkl")

ModuleNotFoundError: No module named 'datasets'